In [1]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F

from torch.nn.utils.rnn import pad_sequence

import logging  

/raid/home/rajivratn/anaconda3/envs/grpo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name="meta-llama/Llama-3.2-1B"

# Initialize the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.eval()
# Freeze LLM parameters
for param in model.parameters():
    param.requires_grad = False
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    

In [10]:
sentences = [
    
    "WHO SHOULD GAIN IT BY A FAIR AND IMPARTIAL ELECTION THEY WERE DETERMIN'D THAT THE MOST VALUABLE POST OF HONOUR IN THE WORLD NAMELY",
    "WHO SHOULD GAIN POST OF HONOUR IN THE WORLD NAMELY IT BY A FAIR AND IMPARTIAL ELECTION THEY WERE DETERMIN'D THAT THE MOST VALUABLE ",
]

In [13]:
with torch.no_grad(): 
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"] # 0 for padding

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    log_probs = F.log_softmax(outputs.logits, dim=-1)  # (B, T, V)

    target_ids = input_ids[:, 1:]
    log_probs = log_probs[:, :-1, :]
    token_log_probs = log_probs.gather(dim=2, index=target_ids.unsqueeze(-1)).squeeze(-1)
    target_mask = attention_mask[:, 1:]
    token_log_probs = token_log_probs * target_mask

rewards = token_log_probs.sum(dim=1) # per-sentence sum of log-probs shape: (B,)
rewards        

tensor([-142.3411, -169.7471])

In [ ]:
(rewards - rewards.mean()) / rewards.std()

tensor([ 0.7071, -0.7071])

: 

In [ ]:
temp